In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

2025-08-02 15:22:00.497026: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-02 15:22:00.657824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754135520.721541   12592 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754135520.739703   12592 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754135520.880562   12592 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
import pandas as pd

def read_data(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=["Content", "Label"])
    df = df.sample(frac=0.2)
    df = df.reset_index(drop=True)  # Optional: reset index after sampling
    return df


In [3]:
path='/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/HateSpeechDatasetBalanced.csv'
df=read_data(path)

In [4]:
X_text = df['Content']
y = df['Label']

In [5]:
# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(
    max_features=4000,      # cap number of features (optional)
    stop_words='english',    # remove common English stopwords
    ngram_range=(1, 2)       # use both unigrams and bigrams
)

In [6]:
#Fit and transform the text data
X_tfidf = vectorizer.fit_transform(X_text)  # Sparse matrix

# Optional: check shape
print("TF-IDF matrix shape:", X_tfidf.shape)


TF-IDF matrix shape: (145224, 4000)


In [16]:
import joblib

# Save the fitted vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer_4000.pkl')


['tfidf_vectorizer_4000.pkl']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y,
    test_size=0.2,       # 20% test, 80% train
    random_state=42,     # for reproducibility
    stratify=y           # optional: ensures class balance in train/test
)

In [11]:
from keras import layers, models, callbacks

model = tf.keras.Sequential([
    layers.Input(shape=(4000,)),

    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),

    layers.Dense(16, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,  # Make sure this is dense, not sparse!
    validation_split=0.2,
    epochs=20,
    batch_size=512,
    callbacks=[early_stop]
)


Epoch 1/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.7467 - loss: 0.6830 - val_accuracy: 0.7877 - val_loss: 0.5404
Epoch 2/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7874 - loss: 0.5446 - val_accuracy: 0.7932 - val_loss: 0.5251
Epoch 3/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.7926 - loss: 0.5339 - val_accuracy: 0.7914 - val_loss: 0.5226
Epoch 4/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.7979 - loss: 0.5251 - val_accuracy: 0.8029 - val_loss: 0.5154
Epoch 5/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.8033 - loss: 0.5213 - val_accuracy: 0.8047 - val_loss: 0.5172
Epoch 6/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.8109 - loss: 0.5175 - val_accuracy: 0.8036 - val_loss: 0.5215
Epoch 7/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.8169 - loss: 0.5151 - val_accuracy: 0.8070 - val_loss: 0.5251


In [12]:
# Accuracy on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


908/908 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8032 - loss: 0.5100
Test Accuracy: 0.8032


In [ ]:
from sklearn.metrics import classification_report

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

print(classification_report(y_test, y_pred))


1362/1362 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
              precision    recall  f1-score   support

           0       0.83      0.78      0.80     21646
           1       0.79      0.84      0.82     21922

    accuracy                           0.81     43568
   macro avg       0.81      0.81      0.81     43568
weighted avg       0.81      0.81      0.81     43568



In [15]:
# Save the model
model.save('/home/abolfazl/Documents/CitizenJournal/citizen_journal/backend/fastapi_backend/HateSpeech/hate_speech_model.keras')


In [13]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.95).astype(int).flatten()


908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [14]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5247030469960406

Classification Report:
               precision    recall  f1-score   support

           0       0.51      1.00      0.68     14459
           1       0.98      0.05      0.10     14586

    accuracy                           0.52     29045
   macro avg       0.75      0.53      0.39     29045
weighted avg       0.75      0.52      0.39     29045

